In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [2]:
def create_server_connection(host_name,user_name,pwd):
    connection=None
    try:
        connection=mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=pwd
        )
        print("MYSQL Database connection successful")
    except Error as err:
        print(f"Error : {err}")
    return connection


host_name='localhost'
user_name='root'             
pwd='12345'
db_name='sql_demo'
        
connection=create_server_connection(host_name,user_name,pwd)  

MYSQL Database connection successful


In [3]:
def create_database(connection,query):
    cursor=connection.cursor()
    try:
        cursor.execute(query)
        print("Database create successfully")
    except Error as err:
        print(f"Error : {err}")

              
create_db_query=f"""create database {db_name}
"""
create_database(connection,create_db_query)

Error : 1007 (HY000): Can't create database 'sql_demo'; database exists


In [4]:
def create_db_connection(host_name,user_name,pwd,db_name):
    connection=None
    try:
        connection=mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=pwd,
            database=db_name
        )
        print("MYSQL Database connection successful")
    except Error as err:
        print(f"Error : {err}")
    return connection

In [5]:
def execute_query(connection,query):
    cursor=connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query Execute Successfully")
    except Error as err:
        print(f"Error : {err}")

In [6]:
create_countrylist_query="""
create table tbl_country_list(
country varchar(30) not null);
"""

create_headerlist_query="""
create table tbl_header(
customer_name varchar(255),
last_consulted_date date,
customer_id varchar(18)
);
"""

create_trailerlist_query="""
create table tbl_trailer(
id varchar(5)
);
"""

connection=create_db_connection(host_name,user_name,pwd,db_name)

execute_query(connection,create_countrylist_query)
execute_query(connection,create_headerlist_query)
execute_query(connection,create_trailerlist_query)

MYSQL Database connection successful
Error : 1050 (42S01): Table 'tbl_country_list' already exists
Error : 1050 (42S01): Table 'tbl_header' already exists
Error : 1050 (42S01): Table 'tbl_trailer' already exists


In [7]:
def insert_countrylist(cntry_name):

    connection=create_db_connection(host_name,user_name,pwd,db_name)
    cursor=connection.cursor()
    
    insert_countrylist_query="INSERT INTO tbl_country_list (country) VALUES (%s)"
        
    val=list([cntry_name])

    try:        
        cursor.execute(insert_countrylist_query,val)
        connection.commit()

        print("Data Insert Successfully")
    except Error as err:
        print(f"Error : {err}")

In [8]:
def read_query(connection,query):
    cursor=connection.cursor()
    try:
        cursor.execute(query)
        results=cursor.fetchall()
        return results
    except Error as err:
        print(f"Error : {err}")

In [9]:
get_countrylist_query="""
select * from tbl_country_list
"""

connection=create_db_connection(host_name,user_name,pwd,db_name)
results=read_query(connection,get_countrylist_query)

for result in results:
    print(result)

MYSQL Database connection successful


In [21]:
def create_countrywise_table(table_name):
    
    create_countrywisetable_query=f"""
    create table {table_name.format()}(
    customer_name varchar(255) not null,
    customer_id varchar(18) not null primary key,
    customer_opendate date,
    last_consulted_date date,
    vaccination_type char(5),
    doctor_consulted varchar(255),
    state char(5),
    country char(30) not null,
    post_code int(5) null,
    date_of_birth date,
    active_customer char(1)
    );
    """
    
    try:        

        connection=create_db_connection(host_name,user_name,pwd,db_name)
        execute_query(connection,create_countrywisetable_query)
        connection.commit()

        print("Country Table Created Successfully")
    except Error as err:
        print(f"Error : {err}")


In [23]:
#insert data into counntrywise table
import datetime

cursor=connection.cursor()

header_list=list([])
record_list=list([])

date_format = "%Y%m%d"
#dt_object = datetime.datetime.strptime(dt_string, date_format)

def input_records():
    connection=create_db_connection(host_name,user_name,pwd,db_name)

    isCountry_Found=False

    with open('demo_data.txt','r') as f:
        for line in f:
                line = line.strip()
                    
                if(line[1]=="T"):
                    for word in line.split("|"):
                        if(word!="T" and word!=" " and len(word)>0):
                            insert_trailer_query="INSERT INTO tbl_trailer (id) VALUES (%s)"
                            val=list([word])

                            cursor.execute(insert_trailer_query,val)
                            connection.commit()
                
                elif(line[1]=="H"):
                    for word in line.split("|"):
                        if(word!="H" and word!=" " and len(word)>0):
                            print(word)
                            header_list.append(word)

                    print(header_list)
                    insert_header_query="INSERT INTO tbl_header VALUES (%s,%s,%s)"
                    cursor.execute(insert_header_query,header_list)
                    connection.commit()
                    header_list.clear()
                
                elif(line[1]=="D"):
                    for word in line.split("|"):
                        if(word!="D" and word!=" " and len(word)>0):
                            print(word)
                            record_list.append(word)
                    
                    print(record_list)
                    
                    #read all country list
                    get_countrylist_query="""
                    select country from tbl_country_list
                    """
                    results=read_query(connection,get_countrylist_query)

                    for result in results:
                        result=','.join(result)
                        if((result.casefold())==(record_list[7].casefold())):
                            isCountry_Found=True        
                            break
                        else:
                            isCountry_Found=False

                    table_name=("tbl_"+record_list[7]).format()
                    print(table_name,type(table_name))
                    
                    if(isCountry_Found==False):
                        insert_countrylist(record_list[7])
                        
                        create_countrywise_table(table_name)
                        
                    record_list[9]=datetime.datetime.strptime(record_list[9], "%d%m%Y").strftime(date_format)

                    insert_countrywise_records_query=f"""
                    INSERT INTO {table_name.format()} 
                            (customer_name,customer_id,customer_opendate,last_consulted_date,
                            vaccination_type,doctor_consulted,state,
                            country,post_code,date_of_birth,active_customer) 
                            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
                    """
                    
                    cursor.execute(insert_countrywise_records_query,record_list)
                    connection.commit()
 
                    record_list.clear()
                
        connection.close()      
#end of method

input_records()

In [ ]:
get_countrylist_query="""
select * from tbl_au
"""

connection=create_db_connection(host_name,user_name,pwd,db_name)
results=read_query(connection,get_countrylist_query)

for result in results:
    print(result)